In [58]:
import pandas as pd

In [59]:
# Import CSV to Pandas
# link = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72495723213&dataTypes=DATE,REPORT_TYPE,WND&startDate=2000-01-01&endDate="
# endDate = "2000-01-02"
# link += endDate
link ="./global-hourly-2023-12-29T11-33-21.csv"
data = pd.read_csv(link)

C:\Users\Lande\AppData\Local\Temp\ipykernel_94948\1697654871.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(link)


In [60]:
# Remove SOD and SOM reports 
data = data[~data['REPORT_TYPE'].isin(['SOD  ','SOM  '])]

In [61]:
# Split the wind data into seperate collumns
split_values = data['WND'].str.split(',', expand=True)
num_columns = len(split_values.columns)
split_values.columns = ["Direction","Direction_Quality","Type","Speed","Speed_Quality"]
data = pd.concat([data, split_values], axis=1)

In [62]:
# Drop redundant collumns
data.drop(["STATION", "SOURCE","SOURCE","CALL_SIGN","QUALITY_CONTROL","DATE.1","REPORT_TYPE.1","REPORT_TYPE","WND"],axis=1,inplace=True)

In [63]:
# Convert Speed and Direction collumns to ints
data['Direction'] = data['Direction'].astype(int)
data['Speed'] = data['Speed'].astype(int)
data['Direction_Quality'] = data['Speed'].astype('string')
data['Speed_Quality'] = data['Speed'].astype('string')
data['Type'] = data['Speed'].astype('string')
print(data.dtypes)

DATE                         object
Direction                     int32
Direction_Quality    string[python]
Type                 string[python]
Speed                         int32
Speed_Quality        string[python]
dtype: object


In [64]:
data.to_csv("output.csv",index=False)

In [65]:
# Standardize calm and variable wind
data.loc[data['Type'] == 'C', 'Speed'] = 0
data.loc[data['Type'] == 'C', 'Direction'] = 0
data.loc[data['Type'] == 'V', 'Direction'] = 999

In [66]:
# Remove data flagged as erroneous
# data = data[data['Type'].isin(['N','C','V'])]
# data = data[data['Direction_Quality'].isin(['0', '1', '4', '5', '9'])]# Direction quality
# data = data[data['Speed_Quality'].isin(['0', '1', '4', '5', '9'])]# Speed quality

In [67]:
# Remove impossibile measurements
data = data[(data['Direction'] == 999) | (data['Direction'].between(0, 360))]

In [68]:
# Remove validation collumns
# data.drop(["Direction_Quality","Type","Speed_Quality"],axis=1,inplace=True)

In [69]:
data.to_csv("output2.csv",index=False)

In [70]:
direction_counts = data['Direction'].value_counts()

print("Direction Counts:")
print(direction_counts)

Direction Counts:
Direction
999    123243
150     14367
160     13812
170     12807
180     11852
140     10404
190      9086
130      6165
330      5815
320      5585
340      5314
200      5226
310      4734
220      4207
350      4162
210      4077
300      3872
250      3587
230      3535
240      3516
120      3444
260      3341
360      3259
290      3237
270      3142
280      2934
10       2373
110      1811
20       1763
30       1145
100      1105
40        787
90        721
50        599
80        549
70        493
60        477
Name: count, dtype: int64


In [71]:
direction_counts = data['Direction_Quality'].value_counts()

print("Direction_Quality Counts:")
print(direction_counts)

Direction_Quality Counts:
Direction_Quality
0       110262
15       45371
21       31262
26       22215
31       16100
36       12997
41       11515
46       10005
51        6766
57        5537
62        3515
67        2198
9999      1612
72        1451
52        1326
77         996
20         801
82         697
88         404
93         282
98         205
56         185
103        134
61         105
108         92
113         66
13          61
22          54
66          52
118         45
71          38
87          34
40          32
35          27
124         22
134         18
129         18
30           6
139          4
149          4
50           4
80           4
92           4
100          3
70           3
123          3
117          2
10           2
60           2
144          2
112          1
160          1
110          1
Name: count, dtype: Int64


In [72]:
direction_counts = data['Type'].value_counts()

print("Type counts:")
print(direction_counts)

Type counts:
Type
0       110262
15       45371
21       31262
26       22215
31       16100
36       12997
41       11515
46       10005
51        6766
57        5537
62        3515
67        2198
9999      1612
72        1451
52        1326
77         996
20         801
82         697
88         404
93         282
98         205
56         185
103        134
61         105
108         92
113         66
13          61
22          54
66          52
118         45
71          38
87          34
40          32
35          27
124         22
134         18
129         18
30           6
139          4
149          4
50           4
80           4
92           4
100          3
70           3
123          3
117          2
10           2
60           2
144          2
112          1
160          1
110          1
Name: count, dtype: Int64


In [73]:
direction_counts = data['Speed'].value_counts()

print("Speed counts:")
print(direction_counts)

Speed counts:
Speed
0       110262
15       45371
21       31262
26       22215
31       16100
36       12997
41       11515
46       10005
51        6766
57        5537
62        3515
67        2198
9999      1612
72        1451
52        1326
77         996
20         801
82         697
88         404
93         282
98         205
56         185
103        134
61         105
108         92
113         66
13          61
22          54
66          52
118         45
71          38
87          34
40          32
35          27
124         22
134         18
129         18
30           6
92           4
80           4
50           4
149          4
139          4
100          3
70           3
123          3
117          2
10           2
60           2
144          2
112          1
160          1
110          1
Name: count, dtype: int64


In [74]:
direction_counts = data['Speed_Quality'].value_counts()

print("Speed_Quality counts:")
print(direction_counts)

Speed_Quality counts:
Speed_Quality
0       110262
15       45371
21       31262
26       22215
31       16100
36       12997
41       11515
46       10005
51        6766
57        5537
62        3515
67        2198
9999      1612
72        1451
52        1326
77         996
20         801
82         697
88         404
93         282
98         205
56         185
103        134
61         105
108         92
113         66
13          61
22          54
66          52
118         45
71          38
87          34
40          32
35          27
124         22
134         18
129         18
30           6
139          4
149          4
50           4
80           4
92           4
100          3
70           3
123          3
117          2
10           2
60           2
144          2
112          1
160          1
110          1
Name: count, dtype: Int64
